In [17]:
import pandas as pd
from datetime import datetime, timedelta

In [22]:
#Lendo base de dados
df_covid = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Trabalhos - Pós Ciência de Dados - UEA/4. Aquisição, pré-processamento e exploração de dados/Trabalho Google Trends + Covid/dados/casos_covid.csv', index_col=0)
#df_trendDiario_br = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Trabalhos - Pós Ciência de Dados - UEA/4. Aquisição, pré-processamento e exploração de dados/Trabalho Google Trends + Covid/dados/sub-dados/1. trends/Trend diário - Brasil.csv')
#df_trendDiario_am = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Trabalhos - Pós Ciência de Dados - UEA/4. Aquisição, pré-processamento e exploração de dados/Trabalho Google Trends + Covid/dados/sub-dados/1. trends/Trend diário - Amazonas.csv')
#df_trendSemanal_br = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Trabalhos - Pós Ciência de Dados - UEA/4. Aquisição, pré-processamento e exploração de dados/Trabalho Google Trends + Covid/dados/sub-dados/1. trends/Trend semanal - Brasil.csv')
#df_trendSemanal_am = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Trabalhos - Pós Ciência de Dados - UEA/4. Aquisição, pré-processamento e exploração de dados/Trabalho Google Trends + Covid/dados/sub-dados/1. trends/Trend semanal - Amazonas.csv')

In [23]:
df_covid

,date,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
1,2020-03-17,894470.0,881935.0,False,False,3,0.33539,2020-03-17,0.0000,0,1,state,AC,3,0
3,2020-03-18,894470.0,881935.0,False,False,3,0.33539,2020-03-18,0.0000,0,2,state,AC,0,0
5,2020-03-19,894470.0,881935.0,False,False,4,0.44719,2020-03-19,0.0000,0,3,state,AC,1,0
7,2020-03-20,894470.0,881935.0,False,False,7,0.78259,2020-03-20,0.0000,0,4,state,AC,3,0
9,2020-03-21,894470.0,881935.0,False,False,11,1.22978,2020-03-21,0.0000,0,5,state,AC,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1655203,2021-02-25,46289333.0,45919049.0,False,False,2014529,4352.03722,2021-02-25,0.0292,58873,367,state,SP,11889,345
1655850,2021-02-26,46289333.0,45919049.0,False,False,2026125,4377.08835,2021-02-26,0.0292,59129,368,state,SP,11596,256
1656497,2021-02-27,46289333.0,45919049.0,False,False,2037267,4401.15869,2021-02-27,0.0292,59428,369,state,SP,11142,299
1657144,2021-02-28,46289333.0,45919049.0,False,False,2041628,4410.57986,2021-02-28,0.0291,59493,370,state,SP,4361,65


In [ ]:
#Selecionando apenas as colunas de interesse do covid
df_covid_brasil = df_covid[df_covid['place_type']=='state'][['date','new_confirmed','new_deaths']]
df_covid_amazonas = df_covid[(df_covid['place_type']=='state') & (df_covid['state']=='AM')][['date','new_confirmed','new_deaths']]

In [ ]:
#Dataset com as colunas de interesse selecionadas
df_covid_brasil

,date,new_confirmed,new_deaths
city,,,
NaN,2020-03-17,3,0
NaN,2020-03-18,0,0
NaN,2020-03-19,1,0
NaN,2020-03-20,3,0
NaN,2020-03-21,4,0
...,...,...,...
NaN,2021-02-23,12077,357
NaN,2021-02-24,12086,329
NaN,2021-02-25,11889,345


### Concatenando os trends e os casos de covid
* Casos diários

In [ ]:
# Agrupando os casos de covid por dia / casos diários
df_covidDiario_br = df_covid_brasil.groupby(by=['date']).sum().reset_index()
df_covidDiario_am = df_covid_amazonas.groupby(by=['date']).sum().reset_index()

In [ ]:
# Casos diários de Covid no Brasil
df_covidDiario_br

,date,new_confirmed,new_deaths
0,2020-02-25,1,0
1,2020-02-26,0,0
2,2020-02-27,0,0
3,2020-02-28,1,0
4,2020-02-29,0,0
...,...,...,...
364,2021-02-23,63588,1358
365,2021-02-24,65387,1446
366,2021-02-25,67783,1567
367,2021-02-26,63888,1349


In [ ]:
# Trends diárias em relação ao Covid
df_trendDiario_br

,date,covid,coronavirus
0,2020-01-01,0,0
1,2020-01-02,0,0
2,2020-01-03,0,0
3,2020-01-04,0,0
4,2020-01-05,0,0
...,...,...,...
412,2021-02-16,68,58
413,2021-02-17,61,53
414,2021-02-18,62,45
415,2021-02-19,69,53


In [ ]:
# Juntando as trends com os casos de covid no mesmo dataframe
df_casosDiario_br = df_trendDiario_br.merge(df_covidDiario_br, how='outer', on='date')
df_casosDiario_am = df_trendDiario_am.merge(df_covidDiario_am, how='outer', on='date')

In [ ]:
#Selecionando os casos de covid até o dia 2021-02-20 (até onde foi coletado as trends)
df_casosDiario_br = df_casosDiario_br[df_casosDiario_br['date']<='2021-02-20']
df_casosDiario_am = df_casosDiario_am[df_casosDiario_am['date']<='2021-02-20']

In [ ]:
df_casosDiario_br

,date,covid,coronavirus,new_confirmed,new_deaths
0,2020-01-01,0.0,0.0,NaN,NaN
1,2020-01-02,0.0,0.0,NaN,NaN
2,2020-01-03,0.0,0.0,NaN,NaN
3,2020-01-04,0.0,0.0,NaN,NaN
4,2020-01-05,0.0,0.0,NaN,NaN
...,...,...,...,...,...
412,2021-02-16,68.0,58.0,65284.0,1092.0
413,2021-02-17,61.0,53.0,47225.0,1189.0
414,2021-02-18,62.0,45.0,50634.0,1439.0
415,2021-02-19,69.0,53.0,55002.0,1331.0


In [ ]:
# Substituindo os valores nulos por zero
df_casosDiario_br = df_casosDiario_br.fillna(0)
df_casosDiario_am = df_casosDiario_am.fillna(0)

In [ ]:
df_casosDiario_br

,date,covid,coronavirus,new_confirmed,new_deaths
0,2020-01-01,0.0,0.0,0.0,0.0
1,2020-01-02,0.0,0.0,0.0,0.0
2,2020-01-03,0.0,0.0,0.0,0.0
3,2020-01-04,0.0,0.0,0.0,0.0
4,2020-01-05,0.0,0.0,0.0,0.0
...,...,...,...,...,...
412,2021-02-16,68.0,58.0,65284.0,1092.0
413,2021-02-17,61.0,53.0,47225.0,1189.0
414,2021-02-18,62.0,45.0,50634.0,1439.0
415,2021-02-19,69.0,53.0,55002.0,1331.0


In [ ]:
# Mudando as colunas do tipo float para int
df_casosDiario_br[['covid','coronavirus','new_confirmed','new_deaths']] = df_casosDiario_br[['covid','coronavirus','new_confirmed','new_deaths']].astype(int)
df_casosDiario_am[['covid','coronavirus','new_confirmed','new_deaths']] = df_casosDiario_am[['covid','coronavirus','new_confirmed','new_deaths']].astype(int)

In [ ]:
# Dataset de casos Diários do Brasil
df_casosDiario_br

,date,covid,coronavirus,new_confirmed,new_deaths
0,2020-01-01,0,0,0,0
1,2020-01-02,0,0,0,0
2,2020-01-03,0,0,0,0
3,2020-01-04,0,0,0,0
4,2020-01-05,0,0,0,0
...,...,...,...,...,...
412,2021-02-16,68,58,65284,1092
413,2021-02-17,61,53,47225,1189
414,2021-02-18,62,45,50634,1439
415,2021-02-19,69,53,55002,1331


In [ ]:
# Dataset dos casos diários do Amazonas
df_casosDiario_am

,date,covid,coronavirus,new_confirmed,new_deaths
0,2020-01-01,0,0,0,0
1,2020-01-02,0,0,0,0
2,2020-01-03,0,0,0,0
3,2020-01-04,0,0,0,0
4,2020-01-05,0,0,0,0
...,...,...,...,...,...
412,2021-02-16,26,32,1808,108
413,2021-02-17,28,25,1808,81
414,2021-02-18,18,13,2141,105
415,2021-02-19,29,13,1950,73


In [ ]:
df_casosDiario_br.to_csv('Covid + Trends diário - Brasil.csv')
df_casosDiario_am.to_csv('Covid + Trends diário - Amazonas.csv')

### Concatenando as trends e os casos de covid
* Casos semanais

In [ ]:
df_trendSemanal_br.iloc[[0,1,2,3,4,-5,-4,-3,-2,-1]]

,date,covid,coronavirus
0,2020-01-05,0,0
1,2020-01-12,0,0
2,2020-01-19,0,4
3,2020-01-26,0,15
4,2020-02-02,0,5
54,2021-01-17,100,3
55,2021-01-24,74,3
56,2021-01-31,72,2
57,2021-02-07,67,2
58,2021-02-14,72,3


In [ ]:
# Transformando coluna de datas para tipo date
df_covidDiario_br['date'] = pd.to_datetime(df_covidDiario_br['date'])
df_covidDiario_am['date'] = pd.to_datetime(df_covidDiario_am['date'])

In [ ]:
'''
Função para agrupar os casos de covid por semana de acordo com as mesmas semanas das trends.
Parâmetros:
  df_casosDiarioCovid: dataframe de casos de covid diário com as colunas |date|new_confirmed|new_death
  df_trendSemanal: dataframe com as trends semanais.
Return:
  Retorna um dataframe já com as trends e casos de covid juntas. Por semana.
'''
def agruparCasosCovidPorSemana(df_casosDiarioCovid, df_trendSemanal) :
  list_dict_covid_casosPorSemana = []
  data_primeiro_caso_covid = df_casosDiarioCovid['date'].min()
  for data in df_trendSemanal['date'] :
    if pd.to_datetime(data) < pd.to_datetime(data_primeiro_caso_covid) :
      list_dict_covid_casosPorSemana.append({'date':data, 'new_confirmed':0, 'new_deaths':0})
    else :
      list_dict_covid_casosPorSemana.append(
          {'date': data, 
          'new_confirmed': df_casosDiarioCovid[(df_casosDiarioCovid['date']>=pd.to_datetime(data)) & (df_casosDiarioCovid['date']<=pd.to_datetime(data)+timedelta(days=6))]['new_confirmed'].sum(),
          'new_deaths': df_casosDiarioCovid[(df_casosDiarioCovid['date']>=pd.to_datetime(data)) & (df_casosDiarioCovid['date']<=pd.to_datetime(data)+timedelta(days=6))]['new_deaths'].sum()
          }
      )
  return pd.DataFrame(list_dict_covid_casosPorSemana)

In [ ]:
df_casosSemanal_br = agruparCasosCovidPorSemana(df_covidDiario_br, df_trendSemanal_br)
df_casosSemanal_am = agruparCasosCovidPorSemana(df_covidDiario_am, df_trendSemanal_am)

In [ ]:
df_casosSemanal_br.iloc[[0,1,2,3,-4,-3,-2,-1]]

,date,new_confirmed,new_deaths
0,2020-01-05,0,0
1,2020-01-12,0,0
2,2020-01-19,0,0
3,2020-01-26,0,0
55,2021-01-24,358806,7492
56,2021-01-31,322189,7116
57,2021-02-07,314475,7573
58,2021-02-14,327047,7345


In [ ]:
df_casosSemanal_br = df_trendSemanal_br.merge(df_casosSemanal_br, how='inner', on='date')
df_casosSemanal_am = df_trendSemanal_am.merge(df_casosSemanal_am, how='inner', on='date')

In [ ]:
df_casosSemanal_br.iloc[[0,1,2,3,-4,-3,-2,-1]]

,date,covid,coronavirus,new_confirmed,new_deaths
0,2020-01-05,0,0,0,0
1,2020-01-12,0,0,0,0
2,2020-01-19,0,4,0,0
3,2020-01-26,0,15,0,0
55,2021-01-24,74,3,358806,7492
56,2021-01-31,72,2,322189,7116
57,2021-02-07,67,2,314475,7573
58,2021-02-14,72,3,327047,7345


In [ ]:
df_casosSemanal_am.iloc[[0,1,2,3,-4,-3,-2,-1]]

,date,covid,coronavirus,new_confirmed,new_deaths
0,2020-01-05,0,0,0,0
1,2020-01-12,0,0,0,0
2,2020-01-19,0,6,0,0
3,2020-01-26,0,22,0,0
55,2021-01-24,72,5,17433,967
56,2021-01-31,56,4,15760,916
57,2021-02-07,48,4,12568,885
58,2021-02-14,38,3,10928,590


In [ ]:
df_casosSemanal_br.to_csv('Covid + Trends semanais - Brasil.csv')
df_casosSemanal_am.to_csv('Covid + Trends semanais - Amazonas.csv')